## Implementation

We need the following components to implement the core word2vec model in tensorflow and as we will see, these components can seen as layers:

1. **The input word embeddings (weights)**: These weights, represented as a matrix $\boldsymbol{W_i}$, transform the onehot encoded word representation $\boldsymbol{i}$ into the distributed wordvector representation $\boldsymbol{v}$.
 
2. **The output word embeddings (set of weights)**: These set of matrices $\boldsymbol{W_{o_1}}, \boldsymbol{W_{o_2}}, ...$, hold the embeddings of words when they are in the context i.e., output. The number depends on the size of the context that we choose for the model.

3. **Softmax**: To transform the dot product scores into probabilities.

4. **Negative log-likelihood/ Cross-entropy loss**: The loss functions for the optimizer.